# Constants

In [1]:
import os, json
from IPython.display import Markdown, display
from dotenv import load_dotenv # requires python-dotenv
from PIL import Image # requires pip install pillow
from datetime import datetime
from common.agents_helper_functions_NEW import *
import importlib.metadata
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

if not load_dotenv("./../config/credentials_my.env"):
    print("Environment variables not loaded, cell execution stopped")
else:
    print("Environment variables have been loaded ;-)")


project_endpoint = os.environ["PROJECT_ENDPOINT"]
deployment_name =  os.environ["MODEL_DEPLOYMENT_NAME"]
api_version = os.environ["OPENAI_API_VERSION"] # at least "2025-03-01-preview"

print(f'Project Endpoint: <{project_endpoint}>')
print(f"azure-ai-projects library installed version: {importlib.metadata.version("azure-ai-projects")}")
print(f"azure-ai-agents library installed version: {importlib.metadata.version("azure-ai-agents")}")

Environment variables have been loaded ;-)
Project Endpoint: <https://mmai-swc-new01-prj-resource.services.ai.azure.com/api/projects/mmai-swc-new01-prj>
azure-ai-projects library installed version: 1.0.0b11
azure-ai-agents library installed version: 1.1.0b1


# [Create AI Foundry Agent Client](https://learn.microsoft.com/en-us/python/api/overview/azure/ai-agents-readme?view=azure-python-preview)
**Note**: I could create the `project` client rather than the `agent` client, however this is easier to read.<br/>
Please consider that `project_client.agens == agents_client`

In [2]:
from azure.ai.agents import AgentsClient
from azure.identity import DefaultAzureCredential

agents_client = AgentsClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),
)

agents_client

# Check if we have any files already on Azure AI Foundry
For better clarity, we delete all pre-existing files, if present

In [3]:
# delete all files

i=0
for file in agents_client.files.list()["data"]:
    i += 1
    agents_client.files.delete(file_id=file.id)
    print(f"{i} - File <{file.filename}> ({file.id}) has been deleted")

print(f"Files deleted: {i}\n")

1 - File <product_info_1.md> (assistant-W37gxZUXm9ViQtAfrADVhE) has been deleted
Files deleted: 1



# Upload file(s) to search into

In [4]:
file = agents_client.files.upload_and_poll(file_path="./product_info_1.md", purpose="assistants")
print(f"Uploaded file: {file}")

Uploaded file: {'object': 'file', 'id': 'assistant-Vtms6WgDc6e3TJDkDY6FTV', 'purpose': 'assistants', 'filename': 'product_info_1.md', 'bytes': 10999, 'created_at': 1748682887, 'expires_at': None, 'status': 'processed', 'status_details': None}


# Create a vector store to vectorize file(s)

In [5]:
vector_store = agents_client.vector_stores.create_and_poll(file_ids=[file.id], name="my_vectorstore")
print(f'Created vector store <{vector_store}>\n\n...and uploaded file "{file.filename}".')

Created vector store <{'id': 'vs_iohVcgQmfeviO3zZyZLLPJh4', 'object': 'vector_store', 'name': 'my_vectorstore', 'status': 'completed', 'usage_bytes': 17143, 'created_at': 1748682890, 'file_counts': {'in_progress': 0, 'completed': 1, 'failed': 0, 'cancelled': 0, 'total': 1}, 'metadata': {}, 'expires_after': None, 'expires_at': None, 'last_active_at': 1748682890}>

...and uploaded file "product_info_1.md".


# Create [FileSearchTool](https://learn.microsoft.com/en-us/python/api/overview/azure/ai-agents-readme?view=azure-python-preview#create-agent-with-file-search) with resources followed by creating agent

In [6]:
from azure.ai.agents.models import FileSearchTool
file_search_tool = FileSearchTool(vector_store_ids=[vector_store.id])
print(f"file_search.definitions: {file_search_tool.definitions}")
print(f"file_search.resources: {file_search_tool.resources}")

file_search.definitions: [{'type': 'file_search'}]
file_search.resources: {'file_search': {'vector_store_ids': ['vs_iohVcgQmfeviO3zZyZLLPJh4']}}


# Create AI Foundry Agent

In [7]:
# using project_client.agents...
ai_agent = agents_client.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name="my-agent",
    instructions="You are helpful agent",
    tools=file_search_tool.definitions,
    tool_resources=file_search_tool.resources,
)

print(f"Created agent, agent ID: {ai_agent.id},\nagent items: {ai_agent.items}")

Created agent, agent ID: asst_bjYll1MatPFVXV1I3cGzJdAX,
agent items: <bound method _MyMutableMapping.items of {'id': 'asst_bjYll1MatPFVXV1I3cGzJdAX', 'object': 'assistant', 'created_at': 1748682892, 'name': 'my-agent', 'description': None, 'model': 'gpt-4o', 'instructions': 'You are helpful agent', 'tools': [{'type': 'file_search'}], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {'file_search': {'vector_store_ids': ['vs_iohVcgQmfeviO3zZyZLLPJh4']}}, 'metadata': {}, 'response_format': 'auto'}>


# Create the thread and attach a new message to it

In [8]:
from azure.ai.agents.models import MessageRole

# Create a thread
thread = agents_client.threads.create()
print(f"Created thread: {thread}\n")

message = agents_client.messages.create(
    thread_id=thread.id,
    role=MessageRole.USER,
    content="Hello, how much for the TrailMaster?",
)

Created thread: {'id': 'thread_JnAPn2AANhatQPrLKGIJnpkb', 'object': 'thread', 'created_at': 1748682892, 'metadata': {}, 'tool_resources': {}}



# Run the agent syncrhonously

In [9]:
%%time

# Run the agent
run = agents_client.runs.create_and_process\
    (thread_id=thread.id, agent_id=ai_agent.id)

print(f"Run finished with status: {run.status}.\n\nRun: {run}")

if run.status == "failed":
    # Check if you got "Rate limit is exceeded.", then you want to get more quota
    print(f"Run failed: {run.last_error}")

Run finished with status: RunStatus.COMPLETED.

Run: {'id': 'run_c3IqGuCgnuUG1at5HtOzeOrh', 'object': 'thread.run', 'created_at': 1748682894, 'assistant_id': 'asst_bjYll1MatPFVXV1I3cGzJdAX', 'thread_id': 'thread_JnAPn2AANhatQPrLKGIJnpkb', 'status': 'completed', 'started_at': 1748682894, 'expires_at': None, 'cancelled_at': None, 'failed_at': None, 'completed_at': 1748682896, 'required_action': None, 'last_error': None, 'model': 'gpt-4o', 'instructions': 'You are helpful agent', 'tools': [{'type': 'file_search'}], 'tool_resources': {}, 'metadata': {}, 'temperature': 1.0, 'top_p': 1.0, 'max_completion_tokens': None, 'max_prompt_tokens': None, 'truncation_strategy': {'type': 'auto', 'last_messages': None}, 'incomplete_details': None, 'usage': {'prompt_tokens': 6043, 'completion_tokens': 35, 'total_tokens': 6078, 'prompt_token_details': {'cached_tokens': 0}}, 'response_format': 'auto', 'tool_choice': 'auto', 'parallel_tool_calls': True}
CPU times: user 12.3 ms, sys: 180 μs, total: 12.4 ms
W

# Fetch messages from the thread after the agent run execution

In [10]:
from azure.ai.agents.models import (MessageTextContent, MessageImageFileContent, MessageTextFileCitationAnnotation, \
    MessageTextFilePathAnnotation, MessageTextUrlCitationAnnotation)

image_files = []
annotations = []
citations = []
citation_annotations = []

if run.status == 'completed':
    messages = agents_client.messages.list(thread_id=thread.id)
    messages_list = list(agents_client.messages.list(thread_id=thread.id))  # Convert iterator to a list
    messages_nr = len(messages_list)
    print(f"Here are the {messages_nr} messages:\n")
    
    for i, message in enumerate(reversed(messages_list), 1):
        j = 0
        print(f"\n===== MESSAGE {i} =====")
        for c in message.content:
            j +=1
            if (type(c) is MessageTextContent):
                print(f"\nMessage {i} / CONTENT {j} (MessageTextContent) --> Text: {c.text.value}")
                for a in c.text.annotations:
                    if type(a) is MessageTextFileCitationAnnotation:
                        print(f">>> Citation in MessageTextContent {j} of message {i}: {a}\n")
                        citations.append(a)
                    elif type(a) is MessageTextFilePathAnnotation:
                        print(f">>> Annotation in MessageTextContent {j} of message {i}: {a}\n")
                        annotations.append(a)
                    elif type(a) is MessageTextUrlCitationAnnotation:
                        print(f">>> Annotation in MessageTextContent {j} of message {i}: {a}\n")
                        citation_annotations.append(a)
                    else:
                        print(f"Unknown type {type(a)}")
            elif (type(c) is MessageImageFileContent):
                print(f"\nMessage {i} / CONTENT {j} (MessageImageFileContent) --> image_file id: {c.image_file.file_id}")
                image_files.append(c.image_file.file_id)
            else:
                print(f"Unknown type {type(a)}")

else:
    print(f"Sorry, I can't proceed because the run status is {run.status}")

Here are the 2 messages:


===== MESSAGE 1 =====

Message 1 / CONTENT 1 (MessageTextContent) --> Text: Hello, how much for the TrailMaster?

===== MESSAGE 2 =====

Message 2 / CONTENT 1 (MessageTextContent) --> Text: The TrailMaster X4 Tent is priced at $250【4:0†source】.
>>> Citation in MessageTextContent 1 of message 2: {'type': 'file_citation', 'text': '【4:0†source】', 'start_index': 41, 'end_index': 53, 'file_citation': {'file_id': 'assistant-Vtms6WgDc6e3TJDkDY6FTV'}}



In [11]:
# Get the last message from the sender
last_msg = list(agents_client.messages.list(thread_id=thread.id))[0]

if last_msg:
    print(last_msg.content[0].text.value)

The TrailMaster X4 Tent is priced at $250【4:0†source】.


# Fetch citations from the messages

In [12]:
print (f"Nr. of file path citations: {len(citations)}\n")

i=0
for a in citations:
    i += 1
    print(f"{i} - citation: {a}")

Nr. of file path citations: 1

1 - citation: {'type': 'file_citation', 'text': '【4:0†source】', 'start_index': 41, 'end_index': 53, 'file_citation': {'file_id': 'assistant-Vtms6WgDc6e3TJDkDY6FTV'}}


# Print citation_annotations from the messages

In [13]:
print (f"Nr. of citation_annotations: {len(annotations)}\n")

i=0
for ca in citation_annotations:
    i += 1
    print(f"{i} - citation_annotations {ca.text}: {ca.url_citation} (from {ca.start_index} to {ca.end_index})")

Nr. of citation_annotations: 0



# Retrieve and download eventual annotations

In [14]:
print (f"Nr. of file path annotations: {len(annotations)}\n")

i=0
for a in annotations:
    i += 1
    print(f"{i} - File annotation paths: {a.text}")
    file_name = a.text.split('/')[-1]
    file_id = a.file_path.file_id

    #agents_client.files.save(file_id=file_id, file_name=file_name)
    agents_client.files.save(file_id=file_id, file_name=file_name)
    print(f"\n>>> file <{file_id}> saved as <{file_name}>")
    
    # project_client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name)
    print(f"File annotation {i} saved as file to: {os.getcwd()}/{file_name}")
    image = mpimg.imread(f"{os.getcwd()}/{file_name}") # read the image
    plt.imshow(image)
    plt.show()

Nr. of file path annotations: 0



# Retrieve eventual citations

In [15]:
print (f"Nr. of file path citations: {len(citations)}\n")

i=0
for a in citations:
    i += 1
    print(f"{i} - citation: {a}")

Nr. of file path citations: 1

1 - citation: {'type': 'file_citation', 'text': '【4:0†source】', 'start_index': 41, 'end_index': 53, 'file_citation': {'file_id': 'assistant-Vtms6WgDc6e3TJDkDY6FTV'}}


# Retrieve and download eventual images

In [16]:
print (f"Nr. of image contents: {len(image_files)}\n")

i=0
# Generate an image file for the bar chart
for image_file in image_files:
    i += 1
    print(f"{i} - Image file id: {image_file}")
    file_name = f"{image_file}.png"
    agents_client.files.save(file_id=image_file, file_name=file_name)
    print(f"Image content {i} file to: {os.getcwd()}/{file_name}")
    image = mpimg.imread(f"{os.getcwd()}/{file_name}") # read the image
    plt.imshow(image)
    plt.show()

Nr. of image contents: 0



# Collect all resources for this project

In [17]:
all_agents = list_all_agents(client=agents_client)
print(all_agents["summary"])

all_threads = list_all_threads(client=agents_client)
print(all_threads["summary"])

all_files = list_all_files(client=agents_client)
print(all_files["summary"])

all_runs = list_all_runs(client=agents_client)
print(all_runs["summary"])

# all_runsteps=list_all_runsteps(project_client)
# print(all_runsteps["summary"])

# all_messages = list_all_messages(project_client)
# print(all_messages["summary"])

all_vectorstores = list_all_vectorstores(client=agents_client)
print(all_vectorstores["summary"])

5 agents
5 threads
1 files
4 runs in 5 threads
1 vector stores


# Teardown for all resources

In [18]:
# delete all vector stores

i=0
for vector_store in all_vectorstores["content"]:
    i += 1
    agents_client.vector_stores.delete(vector_store_id=vector_store.id)
    print(f"{i} - Vector store <{vector_store.id}> has been deleted")

all_vectorstores = list_all_vectorstores(client=agents_client)

print(f"Vector stores deleted: {i}\n")

1 - Vector store <vs_iohVcgQmfeviO3zZyZLLPJh4> has been deleted
Vector stores deleted: 1



In [19]:
# delete all files

i=0
for file in all_files['content']:
    i += 1
    agents_client.files.delete(file_id=file.id)
    print(f"{i} - File <{file.filename}> ({file.id}) has been deleted")

all_files = list_all_files(client=agents_client)

print(f"Files deleted: {i}\n")

1 - File <product_info_1.md> (assistant-Vtms6WgDc6e3TJDkDY6FTV) has been deleted
Files deleted: 1



In [20]:
# delete all threads

i=0
for thread in all_threads["content"]:
    i += 1
    agents_client.threads.delete(thread_id=thread.id)
    print(f"{i} - Thread <{thread.id}> has been deleted")

all_threads = list_all_threads(client=agents_client)

print(f"Threads deleted: {i}\n")

1 - Thread <thread_JnAPn2AANhatQPrLKGIJnpkb> has been deleted
2 - Thread <thread_jg2EetPz79RgvzIueCuffRjO> has been deleted
3 - Thread <thread_Ybb66RPvpOPqRwTx4hlSk6zq> has been deleted
4 - Thread <thread_Z1ey4T85LCumglQoAL9AEkrB> has been deleted
5 - Thread <thread_LoZ3qntfyyjxXYqmFWgHyj9y> has been deleted
Threads deleted: 5



In [21]:
# delete all agents

i=0
for agent in all_agents["content"]:
    i += 1
    agents_client.delete_agent(agent_id=agent.id)
    print(f"{i} - Agent <{agent.id}> has been deleted")

all_agents = list_all_agents(client=agents_client)

print(f"Agents deleted: {i}\n")

1 - Agent <asst_bjYll1MatPFVXV1I3cGzJdAX> has been deleted
2 - Agent <asst_zpDWEyXpy1Q6JPhEDYAIzGpn> has been deleted
3 - Agent <asst_kwp4bA5MptiGPK4vEqcFrb9s> has been deleted
4 - Agent <asst_VDGVN29tQw7dA9vyzClTqdEh> has been deleted
5 - Agent <asst_mq0WeMnh62QU0uZ07NsY7yPA> has been deleted
Agents deleted: 5



# HIC SUNT LEONES